# Spark Context 설정

In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)

# master = namenode
# worker = datanode 원활한 진행을 위해 서로 같은 위치에 둔다.
# hadoop을 부르지 못했기에 나오는 에러

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/02 04:02:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# CSV 파일을 RDD 로딩

In [7]:
filepath = "/home/ubuntu/working/spark-examples/data/restaurant_reviews.csv"

In [8]:
lines = sc.textFile(f"file:///{filepath}")
lines

file:////home/ubuntu/working/spark-examples/data/restaurant_reviews.csv MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
# collect(): rdd 내의 데이터를 모두 출력
lines.collect()
# 문자열!!

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [10]:
# header 제거
header = lines.first() # 첫번쨰 자료

datas = lines.filter(lambda row : row !=header)
datas

PythonRDD[7] at RDD at PythonRDD.scala:53

In [11]:
datas.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [14]:
# 파싱 : 원하는 데이터의 형태로 변경
def parse_task(row):
    # 2번 3번 필드 반환. 단, 3번 필드는 정수로 리턴
    fields = row.split(',')
    
    category = fields[2]
    review_cnt = int(fields[3])
    
    return category, review_cnt


In [15]:
parse_task('0,짜장면,중식,125,')

('중식', 125)

### RDD 내의 모든 데이터에 대해 `parse_task`함수를 적용 후 추출(`map`)

In [16]:
category_reviews = datas.map(parse_task)
category_reviews
# transforamtions

PythonRDD[8] at RDD at PythonRDD.scala:53

In [17]:
category_reviews.collect()
# keyvalue RDD 형태로!
# RDD가 자동으로 셔플한 것임. 같은 것들끼리 모아놓음. 정렬과 다름. 파티션을 배치

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

카테고리 별 리뷰 개수 평균

In [19]:
# 리덕션 필요
# values의 형태 변환
# mapvalues를 주로 사용할 것, map을 사용할 경우 파티션이 무너질 경우가 생김. 
# 셔플이 많이 일어남
category_reviews_count = category_reviews.mapValues(lambda x: (x,1))
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

reduceByKey를 이용해 `Key`별 `Value`에 대한 집계를 수행. 집계 Task정의가 필요하다(함수)

In [23]:
reduced = category_reviews_count.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

예를 들어 중식의 x,y
- 첫 번째 중식: (125,1) -> x
- 두 번째 중식: (235,1) -> y

- 리뷰의 개수끼리 합치기: x[0] + y[0]
- 건수 끼리 합치기: x[1] + y[1]

In [26]:
average = reduced.mapValues(lambda x: x[0]/x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [27]:
sc.stop()

# Transformations
- map()
  - 데이터 마다 배열을 만드는 것
- flatMap()  # 자바 형식이기 때문에 단어와 단어사이를 잇는 문자를 대문자로 치환
  - 평탄화(1차원)된 데이터 배열을 만드는 것
- filter()
- distinct()
- reduceByKey
- groupByKey()
- mapValues()
- flatMapValues()
- sortByKey()

# Actions
- collect()
  - 출력
- count()
- countByValue()
- take()
  - 정해진 개수 출력
- top()
- reduce()
- fold()
- foreach()